In [ ]:
import seaborn as sns
import tensorflow as tf
from cv2 import COLOR_BGR2GRAY, COLOR_GRAY2RGB, createCLAHE, cvtColor, imread, resize
from keras import Model, __version__ as keras_version
from matplotlib.pyplot import ioff, ion, show, style, subplots, tight_layout
from numpy import (
    array,
    bincount,
    clip,
    concatenate,
    float32,
    mean,
    ogrid,
    unique,
    zeros,
)
from numpy.random import normal, random as np_random, randint, seed, uniform
from pandas import DataFrame, read_csv
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from keras import (
    applications,
    callbacks,
    layers,
    mixed_precision,
    optimizers,
    regularizers,
)
from os.path import join
from PIL import Image
from tensorflow import __version__ as tensorflow_version, config, random
from warnings import filterwarnings

filterwarnings("ignore")

In [ ]:
# *** IMPROVED: Mixed precision training for better performance ***
policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

# *** IMPROVED: Better GPU configuration ***
physical_devices = config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    config.experimental.set_memory_growth(physical_devices[0], True)

# Enable eager execution explicitly
config.run_functions_eagerly(False)  # *** IMPROVED: Disabled for better performance ***

# Configuration
style.use("seaborn-v0_8")
seed(42)
random.set_seed(42)

print("TensorFlow version:", tensorflow_version)
print("Keras version:", keras_version)
print("GPU available:", config.list_physical_devices("GPU"))

BASE_PATH = "/home/lipe/BreastCancerClassifier"
JPEG_PATH = join(BASE_PATH, "jpeg")
CSV_PATH = join(BASE_PATH, "csv")

In [ ]:
try:
    train_df = read_csv(join(CSV_PATH, "mass_case_description_train_set.csv"))
    test_df = read_csv(join(CSV_PATH, "mass_case_description_test_set.csv"))

    print("Data loaded successfully")
    print(f"Training set: {len(train_df)} samples")
    print(f"Testing set: {len(test_df)} samples")

except FileNotFoundError:
    print("Error: CSV files for the dataset were not found")
    print("Make sure to download the dataset and adjust the paths")

    # *** IMPROVED: Larger synthetic dataset for better training ***
    train_df = DataFrame(
        {
            "image file path": [
                f"Mass-Training_P_{i:05d}_LEFT_CC" for i in range(2000)
            ],
            "pathology": ["BENIGN" if i % 2 == 0 else "MALIGNANT" for i in range(2000)],
        }
    )

    test_df = DataFrame(
        {
            "image file path": [f"Mass-Test_P_{i:05d}_LEFT_CC" for i in range(400)],
            "pathology": ["BENIGN" if i % 2 == 0 else "MALIGNANT" for i in range(400)],
        }
    )

print("\n=== DATA EXPLORATION ===")
print("Available columns:", train_df.columns.tolist())
print("\nFirst 5 rows of the training set:")
print(train_df.head())

print("\nDistribution of classes in training:")
print(train_df["pathology"].value_counts())

print("\nDistribution of classes in test:")
print(test_df["pathology"].value_counts())

In [ ]:
# Visualization of class distribution
fig, axes = subplots(1, 2, figsize=(12, 5))
train_df["pathology"].value_counts().plot(
    kind="bar", ax=axes[0], color=["skyblue", "lightcoral"]
)
axes[0].set_title("Distribution of Classes - Training")
axes[0].set_xlabel("Pathology")
axes[0].set_ylabel("Number of Samples")

test_df["pathology"].value_counts().plot(
    kind="bar", ax=axes[1], color=["skyblue", "lightcoral"]
)
axes[1].set_title("Distribution of Classes - Test")
axes[1].set_xlabel("Pathology")
axes[1].set_ylabel("Number of Samples")

tight_layout()
show()

In [5]:
# *** IMPROVED: Enhanced image preprocessing for medical images ***
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """
    Enhanced image preprocessing specifically for medical images
    """
    try:
        # Load image
        image = imread(image_path)
        if image is None:
            return None

        # Convert to grayscale first
        gray = cvtColor(image, COLOR_BGR2GRAY)

        # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for better contrast
        clahe = createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)

        # Convert to RGB
        rgb_image = cvtColor(enhanced, COLOR_GRAY2RGB)

        # Resize
        resized = resize(rgb_image, target_size)

        # Normalize to [0,1] range
        normalized = resized.astype(float32) / 255.0

        # Apply additional normalization (z-score normalization)
        normalized = (normalized - normalized.mean()) / (normalized.std() + 1e-8)

        return normalized
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

In [6]:
# *** IMPROVED: Enhanced data generators with medical-specific augmentation ***
def create_data_generators(
    train_df, val_df, test_df, batch_size=16, target_size=(224, 224)
):
    """
    Create enhanced data generators with medical image specific augmentation
    """
    # *** IMPROVED: Medical image specific augmentation ***
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=15,  # Reduced rotation for medical images
        width_shift_range=0.1,  # Reduced shift
        height_shift_range=0.1,  # Reduced shift
        shear_range=0.1,  # Added shear transformation
        zoom_range=0.1,  # Reduced zoom
        horizontal_flip=True,  # Horizontal flip is valid for mammograms
        vertical_flip=False,  # Vertical flip usually not appropriate for mammograms
        brightness_range=[0.8, 1.2],  # Brightness variation
        fill_mode="nearest",
        preprocessing_function=lambda x: (x - x.mean()) / (x.std() + 1e-8),
    )  # Z-score normalization

    # Generator for validation and testing (normalization only)
    val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255,
        preprocessing_function=lambda x: (x - x.mean()) / (x.std() + 1e-8),
    )

    # Create generators
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col="image file path",
        y_col="pathology",
        target_size=target_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
        seed=42,
    )  # Added seed for reproducibility

    val_generator = val_test_datagen.flow_from_dataframe(
        val_df,
        x_col="image file path",
        y_col="pathology",
        target_size=target_size,
        batch_size=batch_size,  # *** IMPROVED: Consistent batch size ***
        class_mode="binary",
        shuffle=False,
        seed=42,
    )

    test_generator = val_test_datagen.flow_from_dataframe(
        test_df,
        x_col="image file path",
        y_col="pathology",
        target_size=target_size,
        batch_size=batch_size,  # *** IMPROVED: Consistent batch size ***
        class_mode="binary",
        shuffle=False,
        seed=42,
    )

    return train_generator, val_generator, test_generator

In [ ]:
# Splitting the training set into training and validation
train_data, val_data = train_test_split(
    train_df, test_size=0.2, random_state=42, stratify=train_df["pathology"]
)

print(f"Training data: {len(train_data)} samples")
print(f"Validation data: {len(val_data)} samples")
print(f"Test data: {len(test_df)} samples")

In [ ]:
# *** IMPROVED: Better parameter configuration ***
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16  # Increased batch size for better training stability
NUM_CLASSES = 2

# *** IMPROVED: Calculate class weights for handling imbalanced data ***
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_df["pathology"])
class_weights = compute_class_weight("balanced", classes=unique(y_encoded), y=y_encoded)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weight_dict}")

In [ ]:
# 4. CONSTRUCTION OF IMPROVED NEURAL NETWORKS
# =============================================================================

print("\n=== BUILDING IMPROVED CNN MODELS ===")


# *** IMPROVED: Enhanced Custom CNN with better architecture ***
def create_improved_custom_cnn(input_shape=(224, 224, 3), num_classes=2):
    """
    Improved Custom CNN with batch normalization, better regularization, and skip connections
    """
    inputs = layers.Input(shape=input_shape)

    # First block
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.15)(x)

    # Second block
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    # Third block with separable convolutions for efficiency
    x = layers.SeparableConv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.SeparableConv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Fourth block
    x = layers.SeparableConv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.SeparableConv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Global Average Pooling instead of Flatten to reduce parameters
    x = layers.GlobalAveragePooling2D()(x)

    # Dense layers with regularization
    x = layers.Dense(
        512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(
        256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Output layer with mixed precision compatibility
    outputs = layers.Dense(1, activation="sigmoid", dtype="float32")(x)

    model = Model(inputs, outputs)
    return model


# *** IMPROVED: Enhanced Transfer Learning Model with fine-tuning strategy ***
def create_improved_transfer_learning_model(input_shape=(224, 224, 3), num_classes=2):
    """
    Improved Transfer Learning model with DenseNet121 and fine-tuning
    """
    # Use DenseNet121 which is excellent for medical imaging
    base_model = applications.DenseNet121(
        include_top=False, weights="imagenet", input_shape=input_shape, pooling=None
    )

    # *** IMPROVED: Freeze initial layers, allow fine-tuning of later layers ***
    for layer in base_model.layers[:-30]:  # Freeze all but last 30 layers
        layer.trainable = False

    inputs = base_model.input
    x = base_model.output

    # Add custom top layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)

    # Multi-scale feature extraction
    x = layers.Dense(
        1024, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(
        512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(
        256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # Output layer with mixed precision compatibility
    outputs = layers.Dense(1, activation="sigmoid", dtype="float32")(x)

    model = Model(inputs, outputs)
    return model

In [10]:
# *** IMPROVED: Additional EfficientNet model for ensemble ***
def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=2):
    """
    EfficientNetB3 based model for ensemble approach
    """
    base_model = applications.EfficientNetB3(
        include_top=False, weights="imagenet", input_shape=input_shape, pooling=None
    )

    # Fine-tuning: unfreeze top layers
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    inputs = base_model.input
    x = base_model.output

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(
        512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(
        256, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(1, activation="sigmoid", dtype="float32")(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
# Create the improved models
model_custom = create_improved_custom_cnn()
model_transfer = create_improved_transfer_learning_model()
model_efficient = create_efficientnet_model()

In [12]:
# *** IMPROVED: Better optimizer configuration ***
def get_optimizer(learning_rate=0.001):
    return optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=1e-4,  # L2 regularization in optimizer
        beta_1=0.9,
        beta_2=0.999,
    )


# *** IMPROVED: Advanced learning rate scheduling ***
def lr_schedule(epoch):
    """
    Learning rate scheduling function
    """
    initial_lr = 0.001
    if epoch < 10:
        return initial_lr
    elif epoch < 20:
        return initial_lr * 0.5
    elif epoch < 30:
        return initial_lr * 0.1
    else:
        return initial_lr * 0.01

In [ ]:
# Compile models with improved configuration
loss = "binary_crossentropy"
metrics = ["accuracy", "precision", "recall", "auc"]

model_custom.compile(optimizer=get_optimizer(0.001), loss=loss, metrics=metrics)

model_transfer.compile(
    optimizer=get_optimizer(0.0005),  # Lower LR for transfer learning
    loss=loss,
    metrics=metrics,
)

model_efficient.compile(optimizer=get_optimizer(0.0005), loss=loss, metrics=metrics)

# Show architectures
print("=== MODEL 1: IMPROVED CUSTOM CNN ===")
model_custom.summary()

print("\n=== MODEL 2: IMPROVED TRANSFER LEARNING (DenseNet121) ===")
model_transfer.summary()

print("\n=== MODEL 3: EFFICIENTNET MODEL ===")
model_efficient.summary()

In [ ]:
# 5. IMPROVED MODEL TRAINING
# =============================================================================
class GraficoAcompanhamento(callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch: list[int] = []
        self.train_acc: list[float] = []
        self.val_acc: list[float] = []
        self.train_loss: list[float] = []
        self.val_loss: list[float] = []

    def on_epoch_end(self, epoch, logs=None):
        self.epoch.append(epoch + 1)
        self.train_acc.append(logs.get("accuracy"))
        self.val_acc.append(logs.get("val_accuracy"))
        self.train_loss.append(logs.get("loss"))
        self.val_loss.append(logs.get("val_loss"))

    def on_train_end(self, logs=None):
        self.fig, self.ax = subplots(ncols=2, figsize=(12, 4))
        ion()

        self.ax[0].clear()
        self.ax[1].clear()

        self.ax[0].plot(self.epoch, self.train_acc, label="Treino")
        self.ax[0].plot(self.epoch, self.val_acc, label="Validação")
        self.ax[0].set_title("Precisão")
        self.ax[0].set_xlabel("Época")
        self.ax[0].set_ylabel("Precisão")
        self.ax[0].legend()
        self.ax[0].grid(True)

        self.ax[1].plot(self.epoch, self.train_loss, label="Treino")
        self.ax[1].plot(self.epoch, self.val_loss, label="Validação")
        self.ax[1].set_title("Perda")
        self.ax[1].set_xlabel("Época")
        self.ax[1].set_ylabel("Perda")
        self.ax[1].legend()
        self.ax[1].grid(True)

        ioff()
        show()


print("\n=== IMPROVED MODEL TRAINING ===")


# *** IMPROVED: Enhanced callbacks ***
def get_callbacks(model_name):
    """
    Get improved callbacks for training
    """
    early_stopping = callbacks.EarlyStopping(
        monitor="val_loss",
        patience=15,  # Increased patience
        restore_best_weights=True,
        verbose=1,
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,  # More aggressive reduction
        patience=7,
        min_lr=1e-7,
        verbose=1,
    )

    lr_scheduler = callbacks.LearningRateScheduler(lr_schedule, verbose=1)

    checkpoint = callbacks.ModelCheckpoint(
        f"best_{model_name}_model.h5",
        monitor="val_accuracy",  # Save best accuracy model
        save_best_only=True,
        verbose=1,
    )

    return [
        early_stopping,
        reduce_lr,
        lr_scheduler,
        checkpoint,
        GraficoAcompanhamento(),
    ]

In [ ]:
# *** IMPROVED: Enhanced synthetic data generation with more realistic patterns ***
def create_improved_synthetic_data(num_samples=2000, image_shape=(224, 224, 3)):
    """
    Create improved synthetic training data with more realistic medical image patterns
    """
    X = np_random((num_samples, *image_shape))

    # Add some structure to make it more realistic
    for i in range(num_samples):
        # Add circular patterns (similar to masses)
        center_x, center_y = randint(50, 174, 2)
        y, x = ogrid[:224, :224]
        mask = (x - center_x) ** 2 + (y - center_y) ** 2 <= randint(20, 50) ** 2
        X[i][mask] *= uniform(0.3, 0.8)  # Darker regions

        # Add noise patterns
        noise = normal(0, 0.1, image_shape)
        X[i] = clip(X[i] + noise, 0, 1)

    # Create labels with some correlation to image features
    y = zeros(num_samples)
    for i in range(num_samples):
        # Create correlation between image features and labels
        mean_intensity = mean(X[i])
        y[i] = 1 if mean_intensity < 0.45 else 0  # Darker images more likely malignant

    return X, y


# Generate improved synthetic data
X_train, y_train = create_improved_synthetic_data(1600)  # Increased training data
X_val, y_val = create_improved_synthetic_data(400)  # Increased validation data
X_test, y_test = create_improved_synthetic_data(400)  # Increased test data

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Class distribution - Training: {bincount(y_train.astype(int))}")
print(f"Class distribution - Validation: {bincount(y_val.astype(int))}")

In [ ]:
# *** IMPROVED: Training with better configuration ***

# Train Model 1: Improved Custom CNN
print("\n--- Training Improved Custom CNN ---")
callbacks_custom = get_callbacks("custom")

history_custom = model_custom.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=50,  # Increased epochs
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,  # Handle class imbalance
    callbacks=callbacks_custom,
    verbose=1,
)

# Train Model 2: Improved Transfer Learning
print("\n--- Training Improved Transfer Learning Model ---")
callbacks_transfer = get_callbacks("transfer")

history_transfer = model_transfer.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=50,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,
    callbacks=callbacks_transfer,
    verbose=1,
)

# Train Model 3: EfficientNet Model
print("\n--- Training EfficientNet Model ---")
callbacks_efficient = get_callbacks("efficient")

history_efficient = model_efficient.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=50,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,
    callbacks=callbacks_efficient,
    verbose=1,
)

In [ ]:
# 6. ENHANCED EVALUATION OF RESULTS
# =============================================================================

print("\n=== ENHANCED EVALUATION OF RESULTS ===")


# *** IMPROVED: Enhanced model evaluation function ***
def evaluate_model_enhanced(model, X_test, y_test, model_name):
    """
    Enhanced model evaluation with more comprehensive metrics
    """
    print(f"\n--- Enhanced Evaluation of {model_name} ---")

    # Predictions
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Comprehensive metrics
    loss, accuracy, precision, recall, auc_score = model.evaluate(
        X_test, y_test, verbose=0
    )

    # Additional metrics
    specificity = confusion_matrix(y_test, y_pred)[0, 0] / (
        confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[0, 1]
    )
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0
    )

    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-Score: {f1_score:.4f}")
    print(f"AUC-ROC: {auc_score:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\nConfusion Matrix:")
    print(cm)

    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=["Benign", "Malignant"]))

    return (
        y_pred_prob,
        y_pred,
        {
            "loss": loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "specificity": specificity,
            "f1_score": f1_score,
            "auc_score": auc_score,
        },
    )

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow import keras


# *** FIXED: Enhanced model evaluation function ***
def evaluate_model_enhanced(model, X_test, y_test, model_name):
    """
    *** FIXED: Enhanced model evaluation with proper error handling ***
    """
    print(f"\n--- Enhanced Evaluation of {model_name} ---")

    try:
        # Predictions
        y_pred_prob = model.predict(X_test, verbose=0)

        # *** FIXED: Handle different prediction shapes ***
        if len(y_pred_prob.shape) > 1 and y_pred_prob.shape[1] > 1:
            # Multi-class output, take the positive class probability
            y_pred_prob = y_pred_prob[:, 1]
        else:
            # Binary output, flatten the array
            y_pred_prob = y_pred_prob.flatten()

        y_pred = (y_pred_prob > 0.5).astype(int)

        # Comprehensive metrics
        loss, accuracy, precision, recall, auc_metric = model.evaluate(
            X_test, y_test, verbose=0
        )

        # *** FIXED: Safer confusion matrix calculation ***
        cm = confusion_matrix(y_test, y_pred)

        # Handle edge cases in confusion matrix
        if cm.shape == (2, 2):
            # Normal case with both classes present
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        elif cm.shape == (1, 1):
            # Only one class predicted
            if unique(y_test).shape[0] == 1:
                # Only one class in true labels too
                specificity = 1.0 if y_pred[0] == y_test[0] else 0.0
            else:
                # One class predicted but two classes in truth
                specificity = 0.0
        else:
            # Unexpected case
            specificity = 0.0
            print(f"⚠️  Unexpected confusion matrix shape: {cm.shape}")

        # *** FIXED: Safer F1 score calculation ***
        f1_score = (
            2 * (precision * recall) / (precision + recall)
            if (precision + recall) > 0
            else 0.0
        )

        print(f"Loss: {loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall (Sensitivity): {recall:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print(f"F1-Score: {f1_score:.4f}")
        print(f"AUC-ROC: {auc_metric:.4f}")

        # Display confusion matrix
        print(f"\nConfusion Matrix:")
        print(cm)

        # *** FIXED: Safer classification report ***
        unique_labels = unique(concatenate([y_test, y_pred]))
        if len(unique_labels) == 2:
            print(f"\nClassification Report:")
            print(
                classification_report(
                    y_test, y_pred, target_names=["Benign", "Malignant"]
                )
            )
        else:
            print(f"\n⚠️  Only one class predicted: {unique_labels}")

        return (
            y_pred_prob,
            y_pred,
            {
                "loss": loss,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "specificity": specificity,
                "f1_score": f1_score,
                "auc_score": auc_metric,
            },
        )

    except Exception as e:
        print(f"❌ Error in model evaluation: {e}")
        # Return dummy values to prevent cascade failures
        dummy_pred_prob = np_random(len(y_test))
        dummy_pred = (dummy_pred_prob > 0.5).astype(int)
        dummy_metrics = {
            "loss": 999.0,
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "specificity": 0.0,
            "f1_score": 0.0,
            "auc_score": 0.5,
        }
        return dummy_pred_prob, dummy_pred, dummy_metrics


# *** FIXED: Improved ensemble prediction function ***
def create_ensemble_predictions(models, X_test):
    """
    *** FIXED: Create ensemble predictions with proper error handling ***
    """
    predictions = []
    successful_models = []

    for i, model in enumerate(models):
        try:
            pred = model.predict(X_test, verbose=0)

            # *** FIXED: Handle different prediction shapes ***
            if len(pred.shape) > 1 and pred.shape[1] > 1:
                # Multi-class output, take positive class probability
                pred = pred[:, 1:2]  # Keep 2D shape for consistency
            elif len(pred.shape) > 1:
                # Already correct shape
                pass
            else:
                # 1D array, reshape to 2D
                pred = pred.reshape(-1, 1)

            predictions.append(pred)
            successful_models.append(i)
            print(f"✅ Model {i+1} prediction shape: {pred.shape}")

        except Exception as e:
            print(f"❌ Error with model {i+1}: {e}")
            continue

    if not predictions:
        print("❌ No successful model predictions!")
        return None, None

    print(f"📊 Successfully got predictions from {len(predictions)} models")

    # *** FIXED: Safer ensemble averaging ***
    try:
        # Stack predictions and average
        predictions_array = np.array(predictions)
        print(f"Predictions array shape: {predictions_array.shape}")

        # Average across models (axis=0)
        ensemble_pred_prob = mean(predictions_array, axis=0)

        # *** FIXED: Ensure correct output shape ***
        if len(ensemble_pred_prob.shape) > 1:
            ensemble_pred_prob = ensemble_pred_prob.flatten()

        ensemble_pred = (ensemble_pred_prob > 0.5).astype(int)

        print(f"Ensemble prediction shape: {ensemble_pred_prob.shape}")
        print(f"Ensemble binary prediction shape: {ensemble_pred.shape}")

        return ensemble_pred_prob, ensemble_pred

    except Exception as e:
        print(f"❌ Error creating ensemble: {e}")
        return None, None


# *** FIXED: Main evaluation section ***
print("\n=== ENHANCED EVALUATION OF RESULTS ===")

# Evaluate all models with error handling
try:
    results_custom = evaluate_model_enhanced(
        model_custom, X_test, y_test, "Improved Custom CNN"
    )
except Exception as e:
    print(f"❌ Error evaluating Custom CNN: {e}")
    results_custom = (
        np.random.random(len(y_test)),
        (np.random.random(len(y_test)) > 0.5).astype(int),
        {
            "loss": 999,
            "accuracy": 0,
            "precision": 0,
            "recall": 0,
            "specificity": 0,
            "f1_score": 0,
            "auc_score": 0.5,
        },
    )

try:
    results_transfer = evaluate_model_enhanced(
        model_transfer, X_test, y_test, "Improved Transfer Learning"
    )
except Exception as e:
    print(f"❌ Error evaluating Transfer Learning: {e}")
    results_transfer = (
        np.random.random(len(y_test)),
        (np.random.random(len(y_test)) > 0.5).astype(int),
        {
            "loss": 999,
            "accuracy": 0,
            "precision": 0,
            "recall": 0,
            "specificity": 0,
            "f1_score": 0,
            "auc_score": 0.5,
        },
    )

try:
    results_efficient = evaluate_model_enhanced(
        model_efficient, X_test, y_test, "EfficientNet Model"
    )
except Exception as e:
    print(f"❌ Error evaluating EfficientNet: {e}")
    results_efficient = (
        np.random.random(len(y_test)),
        (np.random.random(len(y_test)) > 0.5).astype(int),
        {
            "loss": 999,
            "accuracy": 0,
            "precision": 0,
            "recall": 0,
            "specificity": 0,
            "f1_score": 0,
            "auc_score": 0.5,
        },
    )

# *** FIXED: Model Ensemble with error handling ***
print("\n--- Creating Model Ensemble ---")

# Create ensemble
models = [model_custom, model_transfer, model_efficient]
ensemble_result = create_ensemble_predictions(models, X_test)

if ensemble_result[0] is not None:
    ensemble_pred_prob, ensemble_pred = ensemble_result

    # *** FIXED: Safer ensemble evaluation ***
    try:
        ensemble_accuracy = np.mean(ensemble_pred == y_test)
        ensemble_auc = roc_auc_score(y_test, ensemble_pred_prob)

        print("\n--- Ensemble Model Evaluation ---")
        print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
        print(f"Ensemble AUC-ROC: {ensemble_auc:.4f}")

        # *** FIXED: Safer confusion matrix for ensemble ***
        try:
            ensemble_cm = confusion_matrix(y_test, ensemble_pred)
            print("\nEnsemble Confusion Matrix:")
            print(ensemble_cm)
        except Exception as e:
            print(f"❌ Error creating ensemble confusion matrix: {e}")

        # *** FIXED: Safer classification report for ensemble ***
        try:
            unique_labels = np.unique(np.concatenate([y_test, ensemble_pred]))
            if len(unique_labels) == 2:
                print("\nEnsemble Classification Report:")
                print(
                    classification_report(
                        y_test, ensemble_pred, target_names=["Benign", "Malignant"]
                    )
                )
            else:
                print(f"\n⚠️  Ensemble only predicted one class: {unique_labels}")
        except Exception as e:
            print(f"❌ Error creating ensemble classification report: {e}")

    except Exception as e:
        print(f"❌ Error in ensemble evaluation: {e}")
        ensemble_accuracy = 0.0
        ensemble_auc = 0.5

else:
    print("❌ Ensemble creation failed!")
    ensemble_pred_prob = np.random.random(len(y_test))
    ensemble_pred = (ensemble_pred_prob > 0.5).astype(int)
    ensemble_accuracy = 0.0
    ensemble_auc = 0.5

In [ ]:
import tensorflow as tf
from cv2 import (
    CLAHE,
    COLOR_BGR2GRAY,
    COLOR_GRAY2RGB,
    createCLAHE,
    cvtColor,
    resize,
)
from cv2.typing import MatLike
from divisor_de_arquivos import (
    change_dataframe_test_paths,
    change_dataframe_train_paths,
)
from keras import Model
from keras.applications import EfficientNetB3
from keras.callbacks import (
    Callback,
    EarlyStopping,
    LearningRateScheduler,
    ModelCheckpoint,
    ReduceLROnPlateau,
)
from keras.layers import (
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
    Input,
    MaxPooling2D,
    SeparableConv2D,
)
from keras.mixed_precision import Policy, set_global_policy
from keras.models import load_model, Sequential
from keras.optimizers import AdamW
from keras.regularizers import l1_l2
from matplotlib.pyplot import ioff, ion, show, subplots
from numpy import float32, ndarray, uint8, unique
from numpy.random import seed
from pandas import DataFrame, read_csv
from pandas._typing import ArrayLike
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import __version__, config, random, Tensor

# Política de precisão mista para melhorar o desempenho em GPUs compatíveis
set_global_policy(Policy("mixed_float16"))

# habilita alocamento de memória dinâmica quando necessário para GPUs
for gpu in config.list_physical_devices("GPU"):
    config.experimental.set_memory_growth(gpu, True)

# desabilita execução ansiosa para TensorFlow 1.x, (versão 2.x já tem isso desabilitado por padrão)
if int(__version__[0]) < 2:
    config.run_functions_eagerly(False)

seed(42)
random.set_seed(42)

ACTIVATION_FUNCTION: str = "relu"
BATCH_SIZE: int = 16
DROPOUT: float = 0.15
FIELD_SIZE: int = 3
FILTERS: int = 32
IMAGE_DIMENSION: int = 224
KERNEL_REGULARIZER: l1_l2 = l1_l2(l1=1e-5, l2=1e-4)
OPTIMIZER_WEIGHT_DECAY: float = 1e-4
PADDING: str = "same"
UNITS: int = 512
VERBOSE: int = 1


class GraficoAcompanhamento(Callback):
    def on_train_begin(self, logs=None):
        self.epoch: list[int] = []
        self.train_acc: list[float] = []
        self.val_acc: list[float] = []
        self.train_loss: list[float] = []
        self.val_loss: list[float] = []

    def on_epoch_end(self, epoch, logs=None):
        self.epoch.append(epoch + 1)
        self.train_acc.append(logs.get("accuracy"))
        self.val_acc.append(logs.get("val_accuracy"))
        self.train_loss.append(logs.get("loss"))
        self.val_loss.append(logs.get("val_loss"))

    def on_train_end(self, logs=None):
        self.fig, self.ax = subplots(ncols=2, figsize=(12, 4))
        ion()

        self.ax[0].clear()
        self.ax[1].clear()

        self.ax[0].plot(self.epoch, self.train_acc, label="Treino")
        self.ax[0].plot(self.epoch, self.val_acc, label="Validação")
        self.ax[0].set_title("Precisão")
        self.ax[0].set_xlabel("Época")
        self.ax[0].set_ylabel("Precisão")
        self.ax[0].legend()
        self.ax[0].grid(True)

        self.ax[1].plot(self.epoch, self.train_loss, label="Treino")
        self.ax[1].plot(self.epoch, self.val_loss, label="Validação")
        self.ax[1].set_title("Perda")
        self.ax[1].set_xlabel("Época")
        self.ax[1].set_ylabel("Perda")
        self.ax[1].legend()
        self.ax[1].grid(True)

        ioff()
        show()


def learning_rate_schedule(epoch: int) -> float:
    base_learning_rate: float = 0.001
    return (
        base_learning_rate
        if epoch < 10
        else (
            base_learning_rate * 0.5
            if epoch < 20
            else base_learning_rate * 0.1 if epoch < 30 else base_learning_rate * 0.01
        )
    )


def get_callbacks(model_name: str) -> list[Callback]:
    return [
        EarlyStopping(patience=15, restore_best_weights=True, verbose=VERBOSE),
        GraficoAcompanhamento(),
        ModelCheckpoint(
            f"model_fold_{model_name}.keras",
            monitor="val_accuracy",
            verbose=VERBOSE,
            save_best_only=True,
        ),
        LearningRateScheduler(learning_rate_schedule, verbose=VERBOSE),
        ReduceLROnPlateau(factor=0.3, min_lr=1e-7, patience=7, verbose=VERBOSE),
    ]


def preprocess_image(image: ndarray) -> float32:
    image: ndarray = image.astype(uint8)
    gray_scale_image: MatLike = cvtColor(image, COLOR_BGR2GRAY)
    clahe: CLAHE = createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image: MatLike = clahe.apply(gray_scale_image)
    rgb_image: MatLike = cvtColor(enhanced_image, COLOR_GRAY2RGB)
    resized_image: MatLike = resize(rgb_image, IMAGE_SIZE)
    normalized_image: float = resized_image / 255.0
    return (
        (normalized_image - normalized_image.mean()) / (normalized_image.std() + 1e-8)
    ).astype(float32)


def create_efficientnet_model(
    ACTIVATION_FUNCTION=ACTIVATION_FUNCTION,
    DRPOUT=DROPOUT,
    KERNEL_REGULARIZER=KERNEL_REGULARIZER,
    UNITS=UNITS,
) -> Model:
    base_model: Model = EfficientNetB3(
        include_top=False,
        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
        pooling="avg",
    )
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    inputs: list = base_model.input
    output: list = base_model.output

    output: Tensor = Dense(
        UNITS, activation=ACTIVATION_FUNCTION, kernel_regularizer=KERNEL_REGULARIZER
    )(output)
    output: Tensor = Dropout(DRPOUT)(output)
    output: Tensor = BatchNormalization()(output)
    output: Tensor = Dense(
        UNITS * 2, activation=ACTIVATION_FUNCTION, kernel_regularizer=KERNEL_REGULARIZER
    )(output)
    output: Tensor = Dropout(DRPOUT)(output)
    output: Tensor = BatchNormalization()(output)
    output: Tensor = Dense(
        UNITS * 2, activation=ACTIVATION_FUNCTION, kernel_regularizer=KERNEL_REGULARIZER
    )(output)
    output: Tensor = Dropout(DRPOUT)(output)
    output: Tensor = BatchNormalization()(output)
    output: Tensor = Dense(
        UNITS, activation=ACTIVATION_FUNCTION, kernel_regularizer=KERNEL_REGULARIZER
    )(output)
    output: Tensor = Dropout(0.5)(output)
    output: Tensor = BatchNormalization()(output)
    UNITS = 1
    ACTIVATION_FUNCTION = "sigmoid"
    outputs = Dense(units=UNITS, activation=ACTIVATION_FUNCTION, dtype="float32")(
        output
    )

    model: Model = Model(inputs, outputs)
    model.compile(
        optimizer=AdamW(0.005, OPTIMIZER_WEIGHT_DECAY),
        loss="binary_crossentropy",
        metrics=["accuracy", "precision", "recall", "auc"],
    )

    return model


def create_model(
    ACTIVATION_FUNCTION=ACTIVATION_FUNCTION,
    DROPOUT=DROPOUT,
    FIELD_SIZE=FIELD_SIZE,
    FILTERS=FILTERS,
    IMAGE_DIMENSION=IMAGE_DIMENSION,
    KERNEL_REGULARIZER=KERNEL_REGULARIZER,
    PADDING=PADDING,
    UNITS=UNITS,
) -> Sequential:
    inputs = Input(shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3))

    layers = Conv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(inputs)
    layers = BatchNormalization()(layers)
    layers = Conv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D()(layers)
    layers = Dropout(DROPOUT)(layers)

    FILTERS *= 2
    DROPOUT += 0.05

    layers = Conv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(inputs)
    layers = BatchNormalization()(layers)
    layers = Conv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D()(layers)
    layers = Dropout(DROPOUT)(layers)

    FILTERS *= 2
    DROPOUT += 0.05

    layers = SeparableConv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = SeparableConv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D()(layers)
    layers = Dropout(DROPOUT)(layers)

    FILTERS *= 2
    DROPOUT += 0.05

    layers = SeparableConv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = SeparableConv2D(
        FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION, padding=PADDING
    )(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D()(layers)
    layers = Dropout(DROPOUT)(layers)

    layers = GlobalAveragePooling2D()(layers)

    DROPOUT += 0.1

    layers = Dense(
        units=UNITS,
        activation=ACTIVATION_FUNCTION,
        kernel_regularizer=KERNEL_REGULARIZER,
    )(layers)
    layers = BatchNormalization()(layers)
    layers = Dropout(DROPOUT)(layers)

    UNITS //= 2
    DROPOUT -= 0.1

    layers = Dense(
        units=UNITS,
        activation=ACTIVATION_FUNCTION,
        kernel_regularizer=KERNEL_REGULARIZER,
    )(layers)
    layers = BatchNormalization()(layers)
    layers = Dropout(DROPOUT)(layers)

    UNITS = 1
    ACTIVATION_FUNCTION = "sigmoid"

    outputs = Dense(units=UNITS, activation=ACTIVATION_FUNCTION, dtype="float32")(
        layers
    )

    model = Model(inputs, outputs)

    model.compile(
        optimizer=AdamW(0.001, OPTIMIZER_WEIGHT_DECAY),
        loss="binary_crossentropy",
        metrics=["accuracy", "precision", "recall", "auc"],
    )
    return model


IMAGE_SIZE: tuple[int, int] = (IMAGE_DIMENSION, IMAGE_DIMENSION)

train_dataframe: DataFrame = read_csv("csv/mass_case_description_train_set.csv")
test_dataframe: DataFrame = read_csv("csv/mass_case_description_test_set.csv")

# Junta os registros de "BENIGN_WITHOUT_CALLBACK" com "BENIGN"
train_dataframe["pathology"] = train_dataframe["pathology"].replace(
    "BENIGN_WITHOUT_CALLBACK", "BENIGN"
)
train_dataframe["cropped image file path"] = train_dataframe[
    "cropped image file path"
].apply(change_dataframe_train_paths)

test_dataframe["pathology"] = test_dataframe["pathology"].replace(
    "BENIGN_WITHOUT_CALLBACK", "BENIGN"
)
test_dataframe["cropped image file path"] = test_dataframe[
    "cropped image file path"
].apply(change_dataframe_test_paths)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    brightness_range=[0.8, 1.2],
    height_shift_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_image,
    rotation_range=15,
    shear_range=0.1,
    width_shift_range=0.1,
    zoom_range=0.1,
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_image,
)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
models_accuracies: list[tuple[float, float]] = []
sk_fold: StratifiedKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
fold: int = 1
for train, val in sk_fold.split(
    train_dataframe["cropped image file path"], train_dataframe["pathology"]
):
    train_fold = train_dataframe.iloc[train].reset_index(drop=True)
    val_fold = train_dataframe.iloc[val].reset_index(drop=True)

    train_dataset = train_datagen.flow_from_dataframe(
        train_fold,
        batch_size=BATCH_SIZE,
        x_col="cropped image file path",
        y_col="pathology",
        target_size=IMAGE_SIZE,
        class_mode="binary",
        seed=42,
    )

    val_dataset = val_datagen.flow_from_dataframe(
        val_fold,
        batch_size=BATCH_SIZE,
        x_col="cropped image file path",
        y_col="pathology",
        target_size=IMAGE_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=42,
    )

    pathology_encoded: ArrayLike = LabelEncoder().fit_transform(
        train_dataframe["pathology"]
    )
    class_weights: ndarray = compute_class_weight(
        "balanced", classes=unique(pathology_encoded), y=pathology_encoded
    )
    class_weights: dict = dict(enumerate(class_weights))

    model: Sequential = create_model()

    trained_model = model.fit(
        train_dataset,
        batch_size=BATCH_SIZE,
        callbacks=get_callbacks(str(fold)),
        class_weight=class_weights,
        epochs=50,
        validation_data=val_dataset,
        verbose=VERBOSE,
    )
    val_accuracies: tuple[float] = trained_model.history["val_accuracy"]
    max_val_accuracy: float = max(val_accuracies)
    max_val_index: int = val_accuracies.index(max_val_accuracy)
    accuracy: tuple[float, float] = (
        max_val_accuracy,
        trained_model.history["accuracy"][max_val_index],
    )
    models_accuracies.append(accuracy)
    fold += 1

best_accuracy: float = max(models_accuracies)
best_model: int = models_accuracies.index(best_accuracy) + 1
breast_cancer_classifier = load_model(f"model_fold_{best_model}.keras")

In [ ]:
import os
import numpy as np
from keras.utils import load_img, img_to_array
from IPython.display import Image

%store breast_cancer_classifier
%store best_accuracy


diretorio_path = "imagensCancerMama/teste_dataset"
imagens = os.listdir(diretorio_path)
index = np.random.randint(0, len(imagens))
imagem = imagens[index]
imagem_path = os.path.join(diretorio_path, imagem)

print(best_accuracy)

test_image = load_img(imagem_path, target_size=IMAGE_SIZE)
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
resultado = breast_cancer_classifier.predict(test_image)
print(f"Resultado da predição: {resultado[0][0]}")
print(imagem)
print("É um câncer maligno" if resultado[0][0] >= 0.5 else "É um câncer benigno")
Image(filename=imagem_path)